Karina Chiñas Fuentes | 28/02/23

Notes based on the document [Impact of scaling laws on tokamak reactor dimensioning, 2019](https://hal-cea.archives-ouvertes.fr/cea-02378299/document)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

pd.set_option('display.max_columns', None)

From scaling law and other replacements, one can find a relationship that couples the outer radius of the TOKAMAK $R$ to the toroidal magnetic field $B$. I will refer to this relationship as the design equation, since it is widely used when designing TOKAMAKS. The design equation takes the form of

$$
    R^{\gamma_R}\cdot B^{\gamma_B} = \left[\frac{1}{\gamma_{rad}\cdot\left( \frac{1}{\lambda} + \frac{1}{Q} \right)}\right]^{\alpha_p+1}\cdot\frac{C_\beta C_{tr}}{C^{\alpha_n}_{n}C^{\gamma_I}_{I}C_{fus}}\cdot \alpha_0^{-1}\hat{P}^{\alpha_P}_{fus}\cdot M^{\alpha_M}_{eff}\cdot \kappa^{\alpha_\kappa}\cdot \epsilon^{\gamma_\epsilon}\cdot n_N^{\alpha_n}\cdot \beta_N
$$

With

* $\gamma_{rad}$ being the coefficient that parametrizes the total radiative losses in terms of the power heating sources; namely
    $$
        P_{rad} = (1 - \gamma_{rad})P_{scs}, \:\: \text{with}\:\: 0\leq \gamma_{rad} \leq 1
    $$
* $\gamma_I = 1+\alpha_n+\alpha_I$ 
* $\gamma_\epsilon = 1 + \alpha_\epsilon + 2\alpha_I$
* $\gamma_R = \alpha_R + \alpha_I - \alpha_n$
* $\gamma_B = \alpha_B + \alpha_n + \alpha_I + 2$

and constants

* $C_n = 10/\pi \approx 3.1831$
* $C_{tr} = 6\pi^2\cdot10^{19}\cdot10^{-3}\cdot e \approx 0.095$
* $C_I = \mu_0^{-1}2\pi F\cdot10^{-6}$; with F being a plasma shape parameter: $F=\sqrt{ (1+\kappa^2)/2 }$
* $C_\beta = 4\cdot10^{2}\cdot10^{19}\cdot10^3\cdot e \approx 0.805$
* $C_{fus} = E_{fus-particles}\cdot$CONST$\cdot10^{2\cdot19}\cdot\pi^2/2$
    - $E_{fus-particles}$ is the total energy released per fusion reaction. z.B.: DT: $E_{DT} = 17.59$ MeV 
    - **CONST** is a reference on how the reactivity $<\sigma\nu>$ relates to the temperature. If one considers the following plot:
    <img src="REACTIVITYvsTEMP.png" width="350">
    One can get to the relation that
    $$
        <\sigma\nu>_{DT}\:\: \approx 1.1810^{-24}\cdot\hat{T}^2[m^3s^{-1}]
    $$
    Which suggests a range of values for the Temperature of: 10.3 keV to 18.5 keV. One should also consider that the Lawson Criterion is fulfilled. Thus, for the DT case: $C_{fus} \approx 1.6410^{-3}$. By using DT, this would also set $\gamma = 4.94$, since $\gamma = E_{fus-particles}/E_{heating-particle}$; so that, for DT: $E_{DT} / E_{\alpha - DT} = 17.59$ MeV / $3.56$ MeV.  


### Considering Instabilities and Limiting cases:

* Greenwald Limit: $n_N \sim 0.85$
* Normalized Beta Limit: $\beta_N < g$
    - Troyon Limit `g <- 2.8`
    
For simplifying the expression, I will set the following variables:

$$
    \Gamma(Q) = \left[\frac{1}{\gamma_{rad}\cdot\left( \frac{1}{\lambda} + \frac{1}{Q} \right)}\right]^{\alpha_p+1} = \left[\gamma_{rad}\cdot\left( \frac{1}{\lambda} + \frac{1}{Q} \right)\right]^{-(\alpha_p+1)}
$$

and 

$$
    C = \frac{C_\beta C_{tr}}{C^{\alpha_n}_{n}C^{\gamma_I}_{I}C_{fus}} 
$$

So that the design equation takes the form of:

$$
    R^{\gamma_R}\cdot B^{\gamma_B} = C\cdot \Gamma(Q) \cdot \alpha_0^{-1}\hat{P}^{\alpha_P}_{fus}\cdot M^{\alpha_M}_{eff}\cdot \kappa^{\alpha_\kappa}\cdot \epsilon^{\gamma_\epsilon}\cdot n_N^{\alpha_n}\cdot \beta_N
$$

What follows is the analysis of how these parameters influence the toroidal magnetic field and the outer radius. Furthermore, it is possible to consider the following relation which comes from expressing the fusion power in terms of the normalized plasma beta. This leads to:

$$
    R^3B^4 = \frac{\hat{P}_{fus} C^2_{\beta}q^2 }{C_{fus}C^2_I\kappa\epsilon^4}\cdot\beta^2_N
$$

This can help to decouple the design equation. One can also take into consideration the technological limits, like maximum B-field provided by superconductors, and economical considerations. 

In [ ]:
def design_eq(P_fus, Q, scaling_law_params, phys_params, phys_constants):
    α0, αI, αB, αn, αR, αP, ακ, αε, αM = scaling_law_params
    λ, M_eff, κ, ε, n_N, β_N = phys_params     
    Cβ, CI, Cn, C_fus, C_tr  = phys_constants
    
    γI = 1 + αn + αI
    γε = 1 + αε + 2*αI
    γR = αR + αI - αn
    γB = αB + αn + αI + 2
    
    Γ = ( γ_rad*(1/λ + 1/Q) )**( -(αP + 1) )
    C = (Cβ * C_tr) / ( (Cn**αn) * (CI**γI) * C_fus )
    
    RB = C * Γ * (1/α0) * (P_fus**αP) * (M_eff**αM) * (κ**ακ) * (ε**γε) * (n_N**αn) * β_N
    
    return γR, γB, RB